# LAB 5: Image search using CLIP

## Overview
1. The code will run encode all pictures in `datasetFolder_path` into embeddings by `openai-CLIP` pre-trained model
2. Embeddings are store in vectors using `faiss` 
3. The user interface will receive an input image, encode it, and search for similar image in the dataset using `faiss`
4. The dataset image that have the highest similarity score will be shown on the output box.
## Before running the code

1. find the image that is not include in the dataset to test the recommendation algorithm.

## Libraries
1. `torch ftfy regex tqdm` as prerequisite for `openai-clip`
2. `openai-clip` as pre-trained model
3. `numpy` for array manipulation
4. `gradio` for mockup user interface
5. `faiss-cpu` for storing embeddings in vector, search algorithm (for running on cpu)
6. `gdown` for downloading folder shared on google drive

## Setting up workspace for manually selected dataset
If you want to create your own dataset, please follow this guideline,
1. Create an empty folder with the name of `selected_images`
2. In the workspace, the directory should be in this hierachy

```
Workspace Folder
|--/selected_images/
    |---image_01.jpg
    |---image_02.jpg
    |---image_nn.jpg 
|-- 05_CLIP_image_search.ipynb
```

3. uncomment `datasetFolder_path = os.getcwd() + '/selected_images/'`

In [1]:
# install library

! pip install torch ftfy regex tqdm numpy
! pip install openai-clip
! pip install gradio
! pip install faiss-cpu
! pip install gdown

In [24]:
# import essential library

import clip
from PIL import Image
import torch
import os
from tqdm import tqdm
import numpy as np
import faiss

In [25]:
# set running device to cpu

device = "cpu"

In [26]:
# see openai-clip available pre-train model

clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [27]:
# load Vit-B/32 model

model, preprocess = clip.load("ViT-B/32", device=device)

In [ ]:
# download pre-select dataset from shared google drive

import gdown

url = "https://drive.google.com/drive/folders/1eFmm2TUrsPUzPWRP_sE3CnJc6g-La3rC?usp=drive_link"
gdown.download_folder(url, use_cookies=False)

In [28]:
# load dataset

datasetFolder_path = os.getcwd() + '/HM_sample_dataset/' # pre-selected dataset
# datasetFolder_path = os.getcwd() + '/selected_images/' # manually create dataset

In [29]:
# create list of all filename in dataset folder

imageFilename_list = os.listdir(datasetFolder_path)

In [ ]:
# create embeddings vector using FAISS

index = faiss.IndexFlatL2(512) # dimension of 1 embedding decoded from CLIP model is 512

In [30]:
# encode dataset
datasetEmbeddings_list = []

for imageFilename in tqdm(imageFilename_list):
    with torch.no_grad():
        image = preprocess(Image.open(datasetFolder_path + imageFilename)).unsqueeze(0).to(device)
        datasetEmbeddings_list.append(model.encode_image(image).numpy(force=True)[0].astype('float32'))

  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:03<00:00,  5.38it/s]


In [ ]:
# add embeddings into faiss vector

datasetEmbeddings_npArray = np.array(datasetEmbeddings_list) # change to numpy array for FAISS 
index.add(datasetEmbeddings_npArray)

print(index.ntotal) # number of images embeddings store in dataset vector

In [ ]:
import gradio as gr

def recommend_similar_image(imageFile_path):
    print(f"get image path {imageFile_path}")

    test_image = preprocess(Image.open(imageFile_path)).unsqueeze(0).to(device)

    with torch.no_grad():
        test_embeddings = model.encode_image(test_image).numpy(force=True)[0].astype('float32')
        test_embeddings = np.array([test_embeddings])

    recommend_number = 4 # number of recommendations
    square_distance, image_index = index.search(test_embeddings,k)
    print(image_index)
    print(square_distance)
    
    print("Opening Images...")
    recommended_images_list = [(Image.open(datasetFolder_path + imageFilename_list[image_index[0][i]]), f"Recommended Rank {i+1}") for i in range(recommend_number)]
    return recommended_images_list

In [ ]:
demo = gr.Interface(
    fn=recommend_similar_image,
    inputs=gr.Image(type="filepath"),
    outputs= gr.Gallery(),
)

In [ ]:
demo.launch()